In [ ]:
%pip install -U -q pandas dspy datasets

In [ ]:
import dspy  
  
# Initialize ColBERTv2 with a URL to a ColBERT server  
colbert = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')  
  
# Configure DSPy to use this retriever as the default  
dspy.configure(rm=colbert)  
  
# Use the retriever directly  
results = colbert("What is capital of France", k=3)  
  
# Or use it with a DSPy module  
# retrieve = dspy.Retrieve(k=3)  
# results = retrieve("What is retrieval-augmented generation?")

In [ ]:
results

In [ ]:
import dspy  
from dspy.datasets import HotPotQA  
  
# Configure LM and RM  
lm = dspy.LM('openai/gpt-4o-mini')  
colbert = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')  
dspy.configure(lm=lm, rm=colbert)  
  
# Define a simple RAG module  
class RAG(dspy.Module):  
    def __init__(self, num_docs=3):  
        super().__init__()  
        self.retrieve = dspy.Retrieve(k=num_docs)  
        self.generate = dspy.ChainOfThought("context, question -> answer")  
          
    def forward(self, question):  
        passages = self.retrieve(question).passages  
        context = "\n\n".join(passages)  
        print(f"Context: {context}")
        return self.generate(context=context, question=question)  
  
# Use the RAG module  
rag = RAG()  
# result = rag(question="What is capital of France?")  
# result = rag(question="Where is yoga capital of the world?")  
result = rag(question="What is closest park near Google Headquarters?")  
print(result.answer)

In [ ]:
result